In [1]:
import scvelo as scv

import scanpy as sc
from scipy.sparse import issparse

import rpy2.robjects as ro
import anndata2ri

In [2]:
adata = scv.datasets.pancreas()
adata

AnnData object with n_obs × n_vars = 3696 × 27998
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score'
    var: 'highly_variable_genes'
    uns: 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced'
    obsp: 'distances', 'connectivities'

In [3]:
ro.r('library(Seurat)')
ro.r('library(scater)')

R[write to console]: Attaching SeuratObject

R[write to console]: Lade nötiges Paket: SingleCellExperiment

R[write to console]: Lade nötiges Paket: SummarizedExperiment

R[write to console]: Lade nötiges Paket: MatrixGenerics

R[write to console]: Lade nötiges Paket: matrixStats

R[write to console]: 
Attache Paket: ‘MatrixGenerics’


R[write to console]: Die folgenden Objekte sind maskiert von ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, ro

'scater','ggplot2','scuttle',...,'datasets','methods','base'


In [4]:
sc.pp.filter_genes(adata, min_cells=5)

In [5]:
anndata2ri.activate()

In [6]:
ro.globalenv['adata'] = adata

In [7]:
if issparse(adata.X):
    if not adata.X.has_sorted_indices:
        adata.X.sort_indices()

for key in adata.layers:
    if issparse(adata.layers[key]):
        if not adata.layers[key].has_sorted_indices:
            adata.layers[key].sort_indices()

In [8]:
ro.r('seurat_obj = as.Seurat(adata, counts="X", data = NULL)')
ro.r('seurat_obj = RenameAssays(seurat_obj, originalexp = "RNA")')

R[write to console]: Renaming default assay from originalexp to RNA



<rpy2.robjects.methods.RS4 object at 0x7fb8250addc0> [RTYPES.S4SXP]
R classes: ('Seurat',)

In [9]:
ro.r('res <- SCTransform(object=seurat_obj, method = "glmGamPoi", return.only.var.genes = FALSE,)')

R[write to console]: Calculating cell attributes from input UMI matrix: log_umi

R[write to console]: Variance stabilizing transformation of count matrix of size 14915 by 3696

R[write to console]: Model formula is y ~ log_umi

R[write to console]: Get Negative Binomial regression parameters per gene

R[write to console]: Using 2000 genes, 3696 cells



  |======================================================================| 100%


R[write to console]: Found 92 outliers - those will be ignored in fitting/regularization step


R[write to console]: Second step: Get residuals using fitted parameters for 14915 genes



  |======================================================================| 100%


R[write to console]: Computing corrected count matrix for 14915 genes



  |======================================================================| 100%


R[write to console]: Calculating gene attributes

R[write to console]: Wall clock passed: Time difference of 26.88456 secs

R[write to console]: Determine variable features

R[write to console]: Place corrected count matrix in counts slot

R[write to console]: Centering data matrix

  |                                                                            
  |                                                                      |   0%
  |                                                                            
  |====                                                                  |   5%
  |                                                                            
  |=======                                                               |  10%
  |                                                                            
  |==========                                                            |  15%
  |                                                                         

<rpy2.robjects.methods.RS4 object at 0x7fb7e207aec0> [RTYPES.S4SXP]
R classes: ('Seurat',)

In [10]:
norm_x = ro.r('res@assays$SCT@scale.data').T

In [11]:
adata.layers['SCT_normalized_residuals'] = norm_x

In [16]:
norm_x.shape

(3696, 14915)

In [18]:
adata.layers['SCT_corrected_UMI'] = ro.r('res@assays$SCT@counts').T
adata.layers['SCT_lognorm_corrected_UMI'] = ro.r('res@assays$SCT@data').T

In [19]:
adata

AnnData object with n_obs × n_vars = 3696 × 14915
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score'
    var: 'highly_variable_genes', 'n_cells'
    uns: 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced', 'SCT_normalized', 'SCT_corrected_UMI', 'SCT_lognorm_corrected_UMI'
    obsp: 'distances', 'connectivities'

In [20]:
adata.write('data/Pancreas/endocrinogenesis_day15_sctransform.h5ad')